User_intent_parse

In [2]:
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
import json

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

class T1Output(BaseModel):
    product_id: str = Field(..., description="Product/Model ID to be transferred")
    requesting_loc_id: str = Field(..., description="Receiving location ID (Dealer etc.)")
    supplying_loc_id: str = Field(..., description="Sending location ID (Plant, Hub etc.)")

class T2Output(BaseModel):
    product_id: str = Field(..., description="Product/Model ID to be produced")
    requested_qty: int = Field(..., description="Requested production quantity")
    due_date: str = Field(..., description="Requested completion or delivery date")

class T3Output(BaseModel):
    product_id: str = Field(..., description="Product/Model ID")
    target_period: str = Field(..., description="Forecast period (date or week etc.)")
    upcoming_campaigns: str = Field(..., description="Relevant marketing campaigns in period")
    
task_map = {
    "t1": T1Output,
    "t2": T2Output,
    "t3": T3Output,
}

def user_intent_parser(task: str, query: str) -> Dict[str, Any]:
    """
    Parses a natural language user query into a structured output according to the specified task type.
    """
    output_schema = task_map.get(task)
    if output_schema is None:
        raise ValueError(f"Unsupported task.")

    llm_structured = llm.with_structured_output(output_schema)
    result = llm_structured.invoke(query)
    return result.model_dump()

test_questions = {
    "t1": [
        "We need to send the popular 'SNTF-25-CL-AWD' model from the Ulsan factory (P1_ULSAN) to the Gangnam dealer (DEALER_GANGNAM) in Seoul. What is the optimal quantity?",
        "It is reported that the Daegu dealer (DEALER_DAEGU) is low on stock for the 'IONIQ 6 Long Range AWD' model. How many units should be sent from the central hub (HUB_CENTRAL)?",
        "We are planning to transfer 'GRND-35-EX-2WD' sedan stock from the Asan factory (P2_ASAN) to the Incheon dealer (DEALER_INCHEON). Please provide the optimal transfer quantity.",
        "We need to send the 'SNTF-25-CL-AWD' model from the central hub (HUB_CENTRAL) to the Jeju dealer (DEALER_JEJU). What is the best quantity?",
        "The southern logistics center (WAREHOUSE_SOUTH) is almost out of 'GRND-35-EX-2WD' stock. In this situation, what is the best quantity to send to the Busan dealer (DEALER_BUSAN)?",
        "The Gwangju dealer (DEALER_GWANGJU) reports that the 'IONIQ 6 Long Range AWD' is a particularly slow-moving model at their location. They want to replenish stock while avoiding overstocking. What's the best number to send?",
        "The Gangnam dealer (DEALER_GANGNAM) reports that there are 3 units of 'SNTF-25-CL-AWD' stock remaining. How many additional units should be sent from the central hub?",
        "The sedan model 'GRND-35-EX-2WD' is about to have a model year change, and inventory holding costs are expected to surge. How many should be sent to the Daejeon dealer (DEALER_DAEJEON)?",
        "Please send the 'SNTF-25-CL-AWD' model from the Ulsan factory (P1_ULSAN) to the Suwon dealer (DEALER_SUWON).",
        "We need to replenish the 'MODEL-C-EV' stock for 'DEALER_A' from the central hub (HUB_CENTRAL). What quantity would be good?"
    ],
    "t2": [
        "Is it possible to produce 50 units of 'SNTF-25-CL-AWD' by July 28th?",
        "Is it possible to produce 10 units of 'MODEL-C-EV' by July 24th?",
        "Is it possible to produce 500 units of 'SNTF-25-CL-AWD' by August 29th?",
        "Is it possible to produce 1 unit of 'SNTF-25-CL-AWD' by July 25th?",
        "Is it possible to produce 70 units of 'MODEL-C-EV' by August 8th?",
        "Is it possible to produce 2 units of 'SNTF-25-CL-AWD' by July 24th?",
        "Is it possible to produce 5 units of 'MODEL-B-LUXURY' by July 31st?",
        "Is it possible to produce 15 units of 'MODEL-C-EV' by July 25th?",
        "Is it possible to produce 150 units of 'SNTF-25-CL-AWD' by August 1st?",
        "Is it possible to produce 30 units of 'MODEL-C-EV' by July 25th?"
    ],
    "t3": [
        "What is the demand forecast for 'MODEL-B' for next week, considering the 'Summer Special Promotion' is scheduled?",
        "Given the 'New Customer Discount' campaign, what is the final forecast for the 'SNTF-25-CL-AWD' model in 2 weeks?",
        "With the 'EV Subsidy Matching' campaign running, what is the demand forecast for 'MODEL-C-EV' for next week?",
        "What is the forecast for 'MODEL-A-STD' in 3 weeks, when both the 'Year-End Special Discount' and 'Online Purchase Bonus' campaigns will be active?",
        "Considering the 'Quarter-End Sale', what is the demand forecast for 'MODEL-A-STD' for next week?",
        "For 'MODEL-B', a 'Back-to-School Promotion' is planned in 2 weeks. What is the sales forecast?",
        "What is the final demand forecast for 'MODEL-C-EV' for next week, with the 'Inventory Clearance Sale' taking place?",
        "A 'Test Drive Event' is scheduled for 'SNTF-25-CL-AWD' in 3 weeks. What is the expected demand forecast?",
        "What is the forecast for 'MODEL-B' in 2 weeks, considering the 'Summer Special Promotion' will be running?",
        "With the 'New Customer Discount' scheduled, what is the demand forecast for 'MODEL-C-EV' for next week?"
    ]
}

def run_tests():
    for task_name, questions in test_questions.items():
        print(f"\n{'='*20} TESTING TASK: {task_name.upper()} {'='*20}")
        for i, q in enumerate(questions):
            print(f"\n--- Task: {task_name}, Question {i+1}/{len(questions)} ---")
            print(f"  [Input Query]: {q}")
            try:
                structured_output = user_intent_parser(task=task_name, query=q)
                print(f"  [Structured Output]:")
                print(json.dumps(structured_output, indent=4, ensure_ascii=False))
            except Exception as e:
                print(f"  [ERROR]: {e}")
    print(f"\n{'='*20} ALL TESTS COMPLETE {'='*20}")

run_tests()


==================== TESTING TASK: T1 ====================

--- Task: t1, Question 1/10 ---
  [Input Query]: We need to send the popular 'SNTF-25-CL-AWD' model from the Ulsan factory (P1_ULSAN) to the Gangnam dealer (DEALER_GANGNAM) in Seoul. What is the optimal quantity?
  [Structured Output]:
{
    "product_id": "SNTF-25-CL-AWD",
    "requesting_loc_id": "DEALER_GANGNAM",
    "supplying_loc_id": "P1_ULSAN"
}

--- Task: t1, Question 2/10 ---
  [Input Query]: It is reported that the Daegu dealer (DEALER_DAEGU) is low on stock for the 'IONIQ 6 Long Range AWD' model. How many units should be sent from the central hub (HUB_CENTRAL)?
  [Structured Output]:
{
    "product_id": "IONIQ_6_Long_Range_AWD",
    "requesting_loc_id": "DEALER_DAEGU",
    "supplying_loc_id": "HUB_CENTRAL"
}

--- Task: t1, Question 3/10 ---
  [Input Query]: We are planning to transfer 'GRND-35-EX-2WD' sedan stock from the Asan factory (P2_ASAN) to the Incheon dealer (DEALER_INCHEON). Please provide the optimal trans

sales_history_calculator

In [19]:
import psycopg2
import psycopg2.extras
from typing import List, Dict, Any, Optional
from datetime import date

DB_PARAMS = {
    "dbname": "mcp",
    "user": "postgres",
    "password": "1234",
    "host": "localhost",
    "port": "5432"
}

class PostgresConnection:
    def __init__(self, db_params: dict):
        self.db_params = db_params
        self.conn = None

    def __enter__(self):
        self.conn = psycopg2.connect(**self.db_params)
        return self.conn

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.conn:
            self.conn.close()

def calculate_sales_history(product_id: str, location_id: str) -> List[Dict[str, Any]]:
    """
    Calculates the mean and standard deviation of units sold for a specific product 
    at a specific location from the sales history.
    """
    query = """
        SELECT 
            AVG(units_sold) AS mean,
            COALESCE(STDDEV(units_sold), 0) AS standard_deviation
        FROM 
            Sales_History
        WHERE 
            product_id = %s AND location_id = %s;
    """
    params = (product_id, location_id)
    with PostgresConnection(DB_PARAMS) as conn:
        with conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cur:
            cur.execute(query, params)
            return cur.fetchall()
        
print("--- Running Tool Test on Existing DB Data ---")

product = "SNTF-25-CL-AWD"
location = "DEALER_GANGNAM" 

try:
    result = calculate_sales_history(product, location)
    print(f"Tool Output -> {result[0]}")

except Exception as e:
    print(f"Error occured: {e}")

--- Running Tool Test on Existing DB Data ---
Tool Output -> RealDictRow([('mean', Decimal('9.1111111111111111')), ('standard_deviation', Decimal('6.0300175050418478'))])


read_inventory_history

In [ ]:
def read_inventory_history(item_id: str, location_id: str) -> List[Dict[str, Any]]:
    """
    Reads the most recent inventory quantity for a specific item at a specific location.
    """
    query = """
        SELECT 
            quantity_on_hand 
        FROM 
            Inventory_History
        WHERE 
            item_id = %s AND location_id = %s
        ORDER BY 
            snapshot_ts DESC
        LIMIT 1;
    """
    params = (item_id, location_id)
    with PostgresConnection(DB_PARAMS) as conn:
        with conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cur:
            cur.execute(query, params)
            return cur.fetchall()
        
print("--- Running Tool Test on Existing DB Data ---")

item = "SNTF-25-CL-AWD"
location = "P1_ULSAN" 

try:
    result = read_inventory_history(item, location)
    print(f"Tool Output -> {result[0]}")

except Exception as e:
    print(f"Error occured: {e}")

--- Running Tool Test on Existing DB Data ---
Tool Output -> RealDictRow([('quantity_on_hand', 14)])


montecarlo_transfer_optimizer

In [22]:
import numpy as np

def montecarlo_transfer_optimizer(mean: float, standard_deviation: float, quantity_on_hand: int) -> Dict[str, Any]:
    """
    A function that calculates the cost for each inventory quantity and extracts the optimal quantity value that results in the minimum cost.
    """
    min_y = float('inf')
    optimal_q = 0 

    for q in range(0, quantity_on_hand + 1): 
        
        current_y = (q - mean * 10)**2 + standard_deviation * np.cos(q / (mean + 1e-6))
        
        if current_y < min_y:
            min_y = current_y
            optimal_q = q
            
    return {"optimal_transfer_quantity": optimal_q}

print(montecarlo_transfer_optimizer(2.4, 1.14, 500))

{'optimal_transfer_quantity': 24}
